<a href="https://colab.research.google.com/github/LuoYanHao-2025/Data_Science_Group31/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = "/content/drive/MyDrive/WIA1007_Project/smartwatch_health_data.csv"
df = pd.read_csv(file_path)


In [3]:
df.info()

print("\nMissing value")
print(df.isnull().sum())

print("\nUnique Category")
print(df['Activity Level'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User ID                 9799 non-null   float64
 1   Heart Rate (BPM)        9600 non-null   float64
 2   Blood Oxygen Level (%)  9700 non-null   float64
 3   Step Count              9900 non-null   float64
 4   Sleep Duration (hours)  9850 non-null   object 
 5   Activity Level          9800 non-null   object 
 6   Stress Level            9800 non-null   object 
dtypes: float64(4), object(3)
memory usage: 547.0+ KB

Missing value
User ID                   201
Heart Rate (BPM)          400
Blood Oxygen Level (%)    300
Step Count                100
Sleep Duration (hours)    150
Activity Level            200
Stress Level              200
dtype: int64

Unique Category
['Highly Active' 'Highly_Active' 'Actve' 'Seddentary' 'Sedentary' 'Active'
 nan]


In [4]:
df = df.drop(columns=['Heart Rate (BPM)', 'Blood Oxygen Level (%)'], errors='ignore')

In [5]:
df['Sleep Duration (hours)'] = pd.to_numeric(df['Sleep Duration (hours)'], errors='coerce')

In [6]:
Q1 = df['Step Count'].quantile(0.25)
Q3 = df['Step Count'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


In [7]:
df = df[(df['Step Count'] >= lower_bound) & (df['Step Count'] <= upper_bound)]

In [8]:
df['Sleep Duration (hours)'] = df['Sleep Duration (hours)'].fillna(df['Sleep Duration (hours)'].median())
df['Activity Level'] = df['Activity Level'].fillna(df['Activity Level'].mode()[0])
df['Stress Level'] = df['Stress Level'].fillna(df['Stress Level'].mode()[0])


In [9]:
df['Activity Level'] = df['Activity Level'].str.replace('_', ' ').replace({'Actve': 'Active', 'Seddentary': 'Sedentary'})


In [10]:
df = df.drop_duplicates()

In [11]:
df = df.dropna(subset=['User ID'])

In [12]:
df.head()

,User ID,Step Count,Sleep Duration (hours),Activity Level,Stress Level
0,4174.0,5450.390578,7.167236,Highly Active,1
2,1860.0,2826.521994,6.498778,Highly Active,5
3,2294.0,13797.338044,7.367790,Active,3
4,2130.0,15679.067648,6.498778,Highly Active,6
5,2095.0,10205.992256,8.378343,Highly Active,10


In [13]:
output_path = "/content/drive/MyDrive/WIA1007_Project/cleaned_smartwatch_health_data.csv"
df.to_csv(output_path, index=False)

In [14]:
file_path = "/content/drive/MyDrive/WIA1007_Project/Mental_Health_Lifestyle_Dataset.csv"
df2 = pd.read_csv(file_path)

In [15]:
df2 = df2.rename(columns={
    'Exercise Level': 'Activity Level',
    'Sleep Hours': 'Sleep Duration (hours)'
})


In [16]:
df2 = df2[['Activity Level', 'Sleep Duration (hours)', 'Stress Level']]

In [17]:
df2['Sleep Duration (hours)'] = df2['Sleep Duration (hours)'].fillna(df2['Sleep Duration (hours)'].median())
df2['Activity Level'] = df2['Activity Level'].fillna(df2['Activity Level'].mode()[0])
df2['Stress Level'] = df2['Stress Level'].fillna(df2['Stress Level'].mode()[0])
#fill missing value

In [18]:
stress = {'Low': 2, 'Moderate': 5, 'High': 8}
df2['Stress Level'] = df2['Stress Level'].map(stress)

In [19]:
Q1 = df2['Sleep Duration (hours)'].quantile(0.25)
Q3 = df2['Sleep Duration (hours)'].quantile(0.75)
IQR = Q3 - Q1
df2 = df2[(df2['Sleep Duration (hours)'] >= (Q1 - 1.5 * IQR)) & (df2['Sleep Duration (hours)'] <= (Q3 + 1.5 * IQR))]

In [20]:
df2 = df2.drop_duplicates()

In [21]:
df2.head()

,Activity Level,Sleep Duration (hours),Stress Level
0,Low,6.3,2
1,Moderate,4.9,2
2,Low,7.2,8
3,Low,7.2,2
4,Low,7.3,2


In [22]:
output_path = "/content/drive/MyDrive/WIA1007_Project/cleaned_Mental_Health_Lifestyle_Dataset.csv"
df2.to_csv(output_path, index=False)

In [23]:
file_path = "/content/drive/MyDrive/WIA1007_Project/expanded_sleep_data_with_bio.csv"
df3 = pd.read_csv(file_path)

In [24]:
df3 = df3.rename(columns={
    'Physical Activity Level': 'Activity Level',
    'Sleep Duration': 'Sleep Duration (hours)'
})


In [25]:
df3 = df3[['Activity Level', 'Sleep Duration (hours)', 'Stress Level', 'Quality of Sleep']]

In [26]:
for col in df3.columns:
    df3[col] = df3[col].fillna(df3[col].median())

In [27]:
for col in ['Sleep Duration (hours)', 'Activity Level']:
    Q1 = df3[col].quantile(0.25)
    Q3 = df3[col].quantile(0.75)
    IQR = Q3 - Q1
    df3 = df3[(df3[col] >= (Q1 - 1.5 * IQR)) & (df3[col] <= (Q3 + 1.5 * IQR))]

In [28]:
df3 = df3.drop_duplicates()

In [29]:
df3.head()

,Activity Level,Sleep Duration (hours),Stress Level,Quality of Sleep
0,42,6.1,6,6
1,60,6.2,8,6
3,30,5.9,8,4
6,40,6.3,7,6
7,75,7.8,6,7


In [30]:
output_path = "/content/drive/MyDrive/WIA1007_Project/cleaned_expanded_sleep_data_with_bio.csv"
df2.to_csv(output_path, index=False)